# Group 7 Generalization File, Section 4 of the Project

In [ ]:
import pandas as pd
import joblib

# 1) Load serialized preprocessing & models
scaler = joblib.load('../models/scaler.pkl')
cluster_classifier = joblib.load('../models/cluster_clf.pkl')
#model0 = joblib.load('model_cluster0.pkl')
model1 = joblib.load('../models/model_cluster1.pkl')
model2 = joblib.load('../models/model_cluster2.pkl')
#model3 = joblib.load('../models/model_cluster3.pkl')
#features0 = joblib.load('../models/features_cluster0.pkl')
features1 = joblib.load('../models/features_cluster1.pkl')
features2 = joblib.load('../models/features_cluster2.pkl')
#features3 = joblib.load('../models/features_cluster3.pkl')

# 2) Read test set
test_df = pd.read_csv('../data/test_data.csv', index_col=0)

train_transformed = pd.read_csv('../data/train_data_transformed.csv')
cluster_feature_cols = train_transformed.drop(columns=['Cluster','Bankrupt?']).columns.tolist()

# 3) Preprocess & predict cluster-ID
X_test_full = test_df[cluster_feature_cols]
print(X_test_full.info())
X_scaled    = scaler.transform(X_test_full)
cluster_ids = cluster_classifier.predict(X_scaled)

# 4) For each test row, select the right features & model, predict Bankruptcy?
preds = []
for idx, cid in zip(X_test_full.index, cluster_ids):
    if cid == 0:
        print("Missing Model for Cluster 0")
        # X_feat = X_test_full.loc[idx, features0].values.reshape(1, -1)
        # pred   = model0.predict(X_feat)[0]
    elif cid == 1:
        X_feat = X_test_full.loc[idx, features1].values.reshape(1, -1)
        pred = model1.predict(X_feat)[0]
    elif cid == 2:
        X_feat = X_test_full.loc[idx, features2].values.reshape(1, -1)
        pred = model2.predict(X_feat)[0]
    else:  # cid == 3
        print("Missing Model for Cluster 3")
        # X_feat = X_test_full.loc[idx, features3].values.reshape(1, -1)
        # pred   = model3.predict(X_feat)[0]
    preds.append((idx, int(pred)))

# 5) Write submission file
submission = pd.DataFrame(preds, columns=['Index', 'Bankrupt?'])
submission.to_csv('Group7_Generalization.csv', index=False)

print(submission.head())


<class 'pandas.core.frame.DataFrame'>
Index: 1012 entries, 0 to 1011
Data columns (total 19 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0    Cash Flow to Equity                  1012 non-null   float64
 1    Retained Earnings to Total Assets    1012 non-null   float64
 2    Revenue per person                   1012 non-null   float64
 3    Current Liability to Current Assets  1012 non-null   float64
 4    Equity to Liability                  1012 non-null   float64
 5    Cash/Total Assets                    1012 non-null   float64
 6    Total expense/Assets                 1012 non-null   float64
 7    Liability-Assets Flag                1012 non-null   int64  
 8    Total debt/Total net worth           1012 non-null   float64
 9    Operating profit per person          1012 non-null   float64
 10   Quick Assets/Current Liability       1012 non-null   float64
 11   Current Ratio        

NotFittedError: This StackingClassifier instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.